In [20]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import time
import requests
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

Start with Philadelphia, public housing, and all fire incidents 2021.

In [21]:
conn = sqlite3.Connection('./pa_fire_data.db')
df = pd.read_sql("""
    select *,
        cast(substr(ia.INC_DATE, length(ia.INC_DATE) - 3, 4) AS integer) as inc_year
    from basic_incident bi
        join incident_address ia
        using (INCIDENT_KEY)
    where cast(substr(bi.INC_DATE, length(bi.INC_DATE) - 3, 4) AS integer) > 2014
        and cast(substr(ia.INC_DATE, length(ia.INC_DATE) - 3, 4) AS integer) > 2014
        and bi.inc_type <= 200
""", conn)

In [22]:
df.columns = [x.lower() for x in df.columns]

In [23]:
df['address'] = df.replace('None', np.nan, inplace=True)
df['address'] = df[['num_mile', 'street_pre', 'streetname', 'streettype', 'streetsuf', 'apt_no']] \
    .apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

In [24]:
df.address = df.address.str.upper()

In [25]:
public = pd.read_csv('./Public_Housing_Buildings.csv')
public.columns = [x.lower() for x in public.columns]

C:\Users\jackv\AppData\Local\Temp\ipykernel_22004\3050705446.py:1: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  public = pd.read_csv('./Public_Housing_Buildings.csv')


In [26]:
public = public[public.std_st == 'PA']

In [27]:
public['std_addr'] = public.std_addr.str.upper()
public['std_st'] = public.std_st.str.upper()
public['std_city'] = public.std_city.str.upper()

In [28]:
# For the 2015-2019 data
fire_specific = df[(df['inc_type'].isin([111, 113])) & (df.inc_year.astype(int) < 2020)].groupby(['address', 'city', 'zip5', 'inc_type']).size().unstack().reset_index()
fire_specific.columns = ['address', 'city', 'zip5', 'house_fires', 'cooking_fires']
all_fires = df.groupby(['address', 'city', 'zip5']).size().reset_index(name='all_fires')
fire_counts = fire_specific.merge(all_fires, on=['address', 'city', 'zip5'], how='outer')
fire_counts = fire_counts.fillna(0) 
public = public.merge(fire_counts, left_on=['std_addr', 'std_city', 'std_zip5'], right_on=['address', 'city', 'zip5'], how='left').drop(['address', 'city', 'zip5'], axis=1)
public[['house_fires', 'cooking_fires', 'all_fires']] = public[['house_fires', 'cooking_fires', 'all_fires']].fillna(0).astype(int)

# For the 2020-2021 validation data
fire_specific = df[(df['inc_type'].isin([111, 113]))&(df.inc_year.astype(int) >= 2020)].groupby(['address', 'city', 'zip5', 'inc_type']).size().unstack().reset_index()
fire_specific.columns = ['address', 'city', 'zip5', 'house_fires_20-21', 'cooking_fires_20-21']
all_fires = df.groupby(['address', 'city', 'zip5']).size().reset_index(name='all_fires_20-21')
fire_counts = fire_specific.merge(all_fires, on=['address', 'city', 'zip5'], how='outer')
fire_counts = fire_counts.fillna(0) 
public = public.merge(fire_counts, left_on=['std_addr', 'std_city', 'std_zip5'], right_on=['address', 'city', 'zip5'], how='left').drop(['address', 'city', 'zip5'], axis=1)
public[['house_fires_20-21', 'cooking_fires_20-21', 'all_fires_20-21']] = public[['house_fires_20-21', 'cooking_fires_20-21', 'all_fires_20-21']].fillna(0).astype(int)


C:\Users\jackv\AppData\Local\Temp\ipykernel_22004\2222523816.py:14: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  fire_counts = fire_specific.merge(all_fires, on=['address', 'city', 'zip5'], how='outer')


In [29]:
# corrs = []

# # Iterate over numerical columns in pr
# for col in public.select_dtypes(include='number').columns:
#     if col not in ['all_fires', 'block2kx', 'bg2kx', 'place_inc2kx', 
#                    'hlc', 'dpvrc', 'std_zip9', 'dpbc', 'dpbc_cksum', 'std_zip11', 
#                    'c1pprb', 'blkgrp_level', 'exec_dir_fax', 
#                    'national_bldg_id', 'dpvact', 'dpvnost', 'place2kx',
#                    'dpv', 'zcta2kx', 'place_level', 'annl_expns_amnt_prev_yr']:
#         # Get correlation and p-value
#         correlation, p_value = pearsonr(public[col], public['all_fires'])
#         corrs.append((col, correlation, p_value))

# corrs_df = pd.DataFrame(corrs, columns=['columns', 'corr', 'p_value'])
# corrs_df['significant'] = corrs_df['p_value'] < 0.05

NameError: name 'pearsonr' is not defined

In [30]:
public.to_csv('pa_fires_in_ph.csv', index=False)